In [1]:
# Import relevant packages
from math import log
from __future__ import print_function
import pandas as pd
import implicit
import numpy as np
import scipy
import random
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from matplotlib import pyplot as plt
from pylab import savefig
import time 

# Import main dataset
df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
df = df.drop(['Unnamed: 0'], axis=1)

IOError: File lastfm_9000_users.csv does not exist

## Data Preparation|

create_sparse_matrix and calculate_sparsity are functions that allow us to set up the dataset in sparse matrix form.

In [ ]:
# Create sparse matrix from dataframe object
def create_sparse_matrix(data, user_user = True):
    """
    Creates sparse matrix (csr_matrix) out of pandas dataframe.
    
    Parameters: 
    - data: Dataframe of user/artist data
    - user_user: determines whether output will be for user-to-user or item-to-item collaborative filtering
                 if user_user is True, then rows will be items and columns will be users
    
    Returns: 
    - plays_sparse: a sparse csr_matrix
    
    """
    print("Creating sparse matrix...")
    #grab unique users/artist IDS
    users = list(np.sort(data.user_id.unique()))
    artists = list(data.artist_mbid.unique())
    plays = list(data.plays)

    # user-user set-up
    if (user_user == True):
        rows = data.user_id.astype('category', categories=users).cat.codes
        cols = data.artist_mbid.astype('category', categories=artists).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(users),len(artists)))

    #item-item set-up
    else:    
        rows = data.artist_mbid.astype('category', categories=artists).cat.codes
        cols = data.user_id.astype('category', categories=users).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(artists),len(users)))
        
    return plays_sparse

In [ ]:
# Calculate sparsity of mnatrix
def calculate_sparsity(M):
    """
    Computes sparsity of matrix
    
    params:
        M: matrix to be computed
    """
    matrix_size = float(M.shape[0]*M.shape[1]) # Number of possible interactions in the matrix
    num_plays = len(M.nonzero()[0]) # Number of items interacted with
    sparsity = 100*(1 - float(num_plays/matrix_size))
    return sparsity

## Split Train Test

These two functions are used to split the dataset into test/train/tune in various ways. 
make_train_all_user_pairs splits the data into a test and training set with a proportion taken over all users, while split_train_test_per_user makes sure that a certain proportion of each user is held off for the test set. 

split_train_test_per_user also allows for the option of k-fold cross validation. 

In [2]:
# Split train, test using all user-artist pairs
def make_train_all_user_pairs(data, test_pct):
    """
    params:
        data: data set in csr_matrix format
        test_pct: percentage to be test set
    """
    # Create copies of dataset for training and test data
    test = data.copy()
    train = data.copy()
    
    #alter train data, masking/holding-out random user-pair values for some users
    nonzero_idx = train.nonzero() #find indices in data where interaction exists
    
    
    nonzero_pairs = zip(nonzero_idx[0], nonzero_idx[1]) #create pairs of (user, item) index
    
    # Determine how many user-pair values we need to mask, according to test_pct
    random.seed(0) #for reproducibility
    num_samples = int(np.ceil(test_pct * len(nonzero_pairs)))
    
    #Sample random number of user-item pairs without replacement
    samples = random.sample(nonzero_pairs, num_samples) 
    print(type(samples))
    
    # Get user, item row and column indices
    user_idx = [index[0] for index in samples] 
    item_idx = [index[1] for index in samples] 
    
    # Mask the randomly chosen user-item pairs
    train[user_idx,item_idx] = 0 
    # Remove zeros in sparse arrays that was made previously
    train.eliminate_zeros() 
    
    return train, test, list(set(user_idx)), samples #output unique list of user rows that were altered

In [3]:
# Split train, test by user only with interactions#, with test size=total/k
def split_train_test_per_user(data, k, interactions = 20,cross_valid= False):
    """
    Create train matrix with masked values and dictionary of test values 
    
    Parameters:
    - data: csr_matrix, assuming matrix is user-user (item as rows, columns as users)
    - test_pct: percentage of items to mask per user
    
    Output:
    - train: masked matrix
    - test: list of tuples of held out data ((user_idx, item_idx), plays)
    """
    random.seed(0) #for reproducibility
    
    train = data.copy() #transpose to make procedure easier/more intuitive
    
    test = dict() #dict to keep track of masked user-item values
    
    user_count = 0
    test_list=[]
    train_list=[]
    if cross_valid==True: #initialize
        for i in range(k):
            test_list.append(dict())
            train_list.append(train)
    
    for user_idx in tqdm(range(train.get_shape()[0])):

        # Get indices of interactions of this user
        nonzero_idx = train[user_idx].nonzero()

        # Only hold out users that have enough data (greater than interactions #)
        if nonzero_idx[1].shape[0] >= interactions:
            user_count += 1
            # Create list of tuples: interaction index (row, col) with the number of plays
            nonzero_pairs = [((user_idx, item_idx), train[user_idx,item_idx]) for item_idx in nonzero_idx[1]]

            # Sort tuples by descending value
            nonzero_sorted = sorted(nonzero_pairs, key = itemgetter(1), reverse = True)

            # Get top interaction # values, then sample test_pct% randomly from subset
            top_values = nonzero_sorted[0:interactions]

            # Sample random number of item_indexes without replacement
            num_samples = int(np.floor(interactions/float(k)))
            if (cross_valid==False): 
                samples = random.sample(top_values, num_samples) 

                # Append user_idx, item_
                test[user_idx] = [pair[0][1] for pair in samples]

                # Mask the randomly chosen items of this user
                for pair in samples:
                    train[pair[0][0], pair[0][1]] = 0
            
            # Cross Validation Step
            else:
                for i in range(k):
                    train = train_list[i]
                    k_test=test_list[i]
                    random.shuffle(top_values) 
                    samples=top_values[0:num_samples]
                    top_values=top_values[num_samples:]
                    # Append user_idx, item_
                    k_test[user_idx] = [pair[0][1] for pair in samples]
                    test_list[i]=k_test #update test
                    # Mask the randomly chosen items of this user
                    for pair in samples:
                        train[pair[0][0], pair[0][1]] = 0
                    train.eliminate_zeros()
                    # Update train
                    train_list[i]=train 
    if (cross_valid==False):
        # Convert matrix back to initial shape
        return train.T.tocsr(), test, user_count
    else:
        for i in range(k):
            train_list[i]=train_list[i].T.tocsr()
        # Convert matrix back to initial shape
        return train_list, test_list, user_count


In [4]:
# Calculate how many interactions are masked compared to previous dataset
def pct_masked(original, altered):
    altered_n = altered.nonzero()[0].shape[0]
    original_n = original.nonzero()[0].shape[0]
    return (original_n - altered_n)/float(altered_n)

### Baseline Implementation

Below is how we generated our baseline recommendations (taking the most popular artists across the entire dataset and recommending them to everyone)

In [5]:
# Create a baseline model
def baseline(k,user_items):
    plays=user_items.toarray()
    totalplays=np.sum(plays,axis=1)    
    idx = (-totalplays).argsort()[:k]
    return idx

## Evaluation/Metrics

The following are the functions we wrote to determine the NDCG/recall of the baseline, ALS, and KNN recommendations. 

auto_tune_parameter is a function written to determine the best hyperparameters to use for a given model. 

In [6]:
def zeros_list(n):
    listofzeros = [0] * n
    return listofzeros

def dcg_at_k(scores):
    assert scores
    return scores[0] + sum(sc / log(ind, 2) for sc, ind in zip(scores[1:], range(2, len(scores)+1)))

def ndcg_at_k(predicted_scores, user_scores):
    """
    predicted_scores: recommended k items from model
    user_scores: held out items
    """
    assert len(predicted_scores) == len(user_scores)

    idcg = dcg_at_k(sorted(user_scores, reverse=True))
    x = (dcg_at_k(predicted_scores) / idcg) if idcg > 0.0 else 0.0
    
    return x

# Used to evaluate model
def evaluate(model, test, M, n_rec = 20):
    """
    Calculate precision/recall
    
    parameters:
    - model: fitted implicit model that will perform recommendations
    - test: list containing tuples that are heldout for each user
    - M: csr_matrix of item-users, used in fit
    - n_rec: how many recommendations the system outputs
    
    returns:
    - two numpy arrays containing precision and recall
    """
    #TODO: Refactor NDCG and Recall, less dependent on each other
    
    # Transpose matrix to use for implicit's recommend() function
    M_rec = M.T.tocsr() 
    
    tp = float(0)
    test_n = float(0)
    print('Evaluating model...')
    
    ndcg = list()
    
    # Calculate true positives for each user, append results to list
    for user, holdout_items in tqdm(test.items()):
        
        # Get list of item recs for each user
        rec = model.recommend(user, M_rec, N=n_rec, filter_already_liked_items=True) #returns (item_id, score)
        rec_items = [pair[0] for pair in rec] #get only item_id
        test_n += len(holdout_items) #total number of heldout items
        
        # For NDCG
        predicted_scores = zeros_list(n_rec)
        user_scores = zeros_list(n_rec)
        i = 0
            
        # Count true positives in recommended items
        for item in holdout_items: 
            value = M[user,item]
            user_scores[i] = value
            i += 1
            if item in rec_items:
                predicted_scores[rec_items.index(item)] = value #if holdout items is in recommended
                tp += 1
        # Calculate NDCG
        ndcg.append(ndcg_at_k(predicted_scores, user_scores))

    recall = tp/test_n
    return recall, np.mean(ndcg)

In [7]:
# Used to evaluate baseline model
def evaluate_base(rec_items, test, M, n_rec = 20):
    """
    Calculate recall
    
    parameters:
    - model: fitted implicit model that will perform recommendations
    - test: list containing tuples that are heldout for each user
    - M: csr_matrix of item-users, used in fit
    - n_rec: how many recommendations the system outputs
    returns:
    - numpy array containing recall
    """
    # Transpose matrix to use for implicit's recommend() function
    M_rec = M.T.tocsr()
    
    tp = float(0)
    test_n = float(0)
    rec_items = list(rec_items)
    ndcg = list()

    print('Evaluating model...')
    # Calculate true positives for each user, append results to list

    for user, holdout_items in tqdm(test.items()):
        test_n += len(holdout_items)
        # Count true positives in recommended items
        predicted_scores = zeros_list(n_rec)
        user_scores = zeros_list(n_rec)
        i = 0
        for item in holdout_items:
            value = M[user,item]
            user_scores[i] = value
            i += 1
            if item in rec_items:
                predicted_scores[rec_items.index(item)] = value #if holdout items is in recommended
                tp += 1

        ndcg.append(ndcg_at_k(predicted_scores, user_scores))
        
    recall = tp/test_n
    return recall,np.mean(ndcg)

In [8]:
"""
Function that identifies optimal parameter values given relevant models and arrays of parameters

input: 
    - k: # of folds within the training set (split into tuning sets)
    - interactions: size of recommendation list
    - model: model that is being optimized   
    - data: sparse user-item matrix
    - param1: list of values to try for hyperparameter 1.
    - param2: list of values to try for hyperparameter 2. 
    - param3: list of values to try for hyperparameter 3.
    NOTE: if using KNN, there are 3 parameters. param2 is the integer used for k1; 
    only k and b are tested only param1 and param3 should be lists.
    If using ALS, there are 2 parameters. both should be lists. 
    
output:
    - max_ndcg_list: a list of k tuples, one for each fold. 
        each tuple is in the form (max_ndcg,max_first_param,max_second_param,max_recall)
        which records the best ndcg, and the two params that achieved it, 
        and the max_recall achieved (which may be from different param values).
    - heatmap_list: a list of k heatmaps of the NDCG values for the two tested 
        parameters (one heatmap per fold). Useful for visualizations
"""
def auto_tune_parameter(k,interactions,model,data,param1,param2,param3=None):
    # Train model
    # Create list of MAX NDCG and Recall depending on # params
    max_ndcg_list=[] #will end up being length k list of tuples of best param values
    heatmap_list=[]
    if param3==None: #for ALS
        num_param=2
        first_param=param1
        second_param=param2
    else: #for KNN
        num_param=3
        first_param=param1
        second_param=param3
    train_and_tune,test,user_count=split_train_test_per_user(data, k+1, interactions,cross_valid= False)
    train_list, tune_list, user_count = split_train_test_per_user(train_and_tune.T.tocsr(),k,int(np.ceil(((k-1)/k)*interactions)),cross_valid=True)
    #to be updated via max: to determine final params to use on test set 
    test_ndcg=0
    test_first_param=first_param[0]
    test_second_param=second_param[0]
    #create recall/NDCG matrix storing for each combination of 
    for fold in range(k): #For each fold; there are k-1 folds within train_and_tune
        ndcg_heatmap=[[0 for x in range(len(second_param))] for y in range(len(first_param))]
        print(ndcg_heatmap)
        train=train_list[fold]
        tune=tune_list[fold]
        max_first_param=first_param[0] #initialize best value of first_param for this fold
        max_second_param=second_param[0] #initialize best value of second_param for this fold
        max_recall=0
        max_ndcg=0
        value1_index=0 #index for heatmap
        print("Fitting fold number...",fold)
        for value1 in first_param:
            value2_index=0
            for value2 in second_param:
                if num_param==2:
                    print("Trying ",(value1,value2))
                    usemodel=model(value1,value2)
                if num_param==3:
                    print("Trying ",(value1,param2,value2))
                    usemodel=model(value1,param2,value2)
                usemodel.fit(train, show_progress=False)
                recall,ndcg = evaluate(usemodel,tune,data)
                print(value1_index,value2_index)
                ndcg_heatmap[value1_index][value2_index]=ndcg #update heatmap 
                #recall_list.append(recall)
                #update maximum values
                max_recall=max(max_recall,recall)
                if ndcg>max_ndcg: 
                    max_ndcg=ndcg
                    max_first_param=value1
                    max_second_param=value2
                value2_index=value2_index+1
            value1_index=value1_index+1    
        max_ndcg_list.append([max_ndcg,max_first_param,max_second_param,max_recall])
        if max_ndcg>test_ndcg:
            print("Fold ",fold," beat the record for ncdg!")
            print("New best ndcg is ",max_ndcg)
            print("New best params are ",(max_first_param,max_second_param))
            test_ndcg=max_ndcg
            test_first_param=max_first_param
            test_second_param=max_second_param
        heatmap_list.append(ndcg_heatmap)
        print("end of fold---------------------------")

    #Now, test_first_param and test_second_param should be optimized
    if num_param==2:
        usemodel=model(test_first_param,test_second_param)
    if num_param==3:
        usemodel=model(test_first_param,param2,test_second_param)
    usemodel.fit(train_and_tune,show_progress=True)
    final_recall,final_ndcg = evaluate(usemodel,test,data)
    
    print("The recall on the test set is ", final_recall,", after hyperparameter optimization")
    print("The ndcg on the test set is ",final_ndcg,", after hyperparameter optimization")
    
    return max_ndcg_list,heatmap_list 

# Main Script

#### 1. Setup: Create sparse matrix

In [10]:
#create sparse matrix
plays_sparse = create_sparse_matrix(df).astype('float')
print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

NameError: name 'create_sparse_matrix' is not defined

In [13]:
# Split data into training and test sets
train, test, user_count = split_train_test_per_user(plays_sparse, 3, 10)
print("Percentage of original data masked:", pct_masked(plays_sparse, train))
print("Users masked:", user_count)


Percentage of original data masked: 0.06548419166887459
Users masked: 8980


#### Model-Based (ALS)

Here we run the model-based ALS Matrix Factorization and display the results. 

In [14]:
model = implicit.als.AlternatingLeastSquares(50)

# Train model
print("Fitting model...")
model.fit(train, show_progress=True)

# Compute and output recall and NDCG
recall, ndcg = evaluate(model, test, plays_sparse)
print("Recall:",recall*100,'%')
print("Average NDCG:",ndcg*100,'%')

  0%|          | 0/15 [00:00<?, ?it/s]

Fitting model...


100%|██████████| 15.0/15 [00:05<00:00,  2.66it/s]

Evaluating model...



Recall: 22.178916109873796 %
Average NDCG: 12.66723278283797 %


#### K-Nearest Neighbors (Item-Based)

Here we run the KNN and display the results.

In [15]:
model_knn = implicit.nearest_neighbours.BM25Recommender()

# Train Model
print("Fitting model...")
model_knn.fit(train, show_progress=True)

# Compute and output recall and NDCG
recall, ndcg = evaluate(model_knn, test, plays_sparse)
print("Recall:",recall*100,'%')
print("Average NDCG:",ndcg*100,'%')

  0%|          | 0/47102 [00:00<?, ?it/s]

Fitting model...


100%|██████████| 47102/47102 [00:00<00:00, 86340.16it/s]


Evaluating model...



Recall: 3.103192279138827 %
Average NDCG: 1.6345120853324613 %


#### Baseline

Here we run baseline and display the results.

In [ ]:
# Set up baseline model
user_items = plays_sparse.T.tocsr()
rec_items=baseline(20,user_items)

# Compute and output recall and NDCG
recall,ndcg = evaluate_base(rec_items,test,plays_sparse)
print("Recall:",recall*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

# Cross Validation and Parameter Tuning (k-fold)

In this section, we use k-fold cross validation to tune hyperparameters and evaluate our models.

### Splitting into test and training sets

In [16]:
# Cross Validation test
k=5
train_list, test_list, user_count = split_train_test_per_user(plays_sparse,k,20,cross_valid=True)

## Evaluate and tune ALS

Tuning two parameters: number of latent factors and the regularization factor. 

For latent factors, we try values [10,20,30,40,50,60]

For regularization factors, we try [.01,.03,.05,.07]

In [ ]:
start = time.time()
model=implicit.als.AlternatingLeastSquares
ndcg_list,heatmap_list=auto_tune_parameter(4,20,model,plays_sparse,[10,20,30,40,50,60],[.01,.03,.05,.07],param3=None)
stop = time.time()
total = stop-start

In [ ]:
# Plot heatmap of parameter tuning results
sns.set_style("whitegrid")
sns.heatmap(heatmap_list[2], 
            xticklabels=['0.01','0.03','0.05','0.07'], 
            yticklabels=['10','20','30','40','50','60'], 
            cbar_kws={'label':'NDCG Score'})
plt.ylabel("Number of Latent Factors")
plt.xlabel("Regularization Factor")
plt.title("ALS NDCG scores according to Model Parameters")
plt.show()

Here we analyze the scalability of ALS by looking at datasets with 9k, 20k, 60k, and 150k users:

In [ ]:
# The following block imports larger datasets for scaling tests, these expanded CSVs are not available in the repo
files9k = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
files20k = pd.read_csv('lastfm_20k_users.csv', na_filter=False)
files60k = pd.read_csv('lastfm_60k_users.csv', na_filter=False)
files150k = pd.read_csv('lastfm_150k_users.csv', na_filter=False)
files40k = get_users(files150k, 40000)
files = [files9k, files20k, files40k, files60k]

In [ ]:
# Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
size = [9000, 20000, 40000, 60000]
ndcg_size = []
recall_size = []
for i in files:
    model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)

    #create sparse matrix
    plays_sparse = create_sparse_matrix(i).astype('float')
    print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

    train, test, user_count = split_train_test_per_user(plays_sparse, 4, 20)

    # train model 
    print("Fitting model...")
    model.fit(train, show_progress=True)

    recall, ndcg = evaluate(model, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_size.append(recall)
    ndcg_size.append(ndcg)

In [ ]:
# Plot scalability of ALS model
ndcg_size_df = pd.DataFrame({'N':size,
                       'NDCG': ndcg_size})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_size_df)
plt.title('NDCG by Input Size for ALS')
plt.xlabel('Number Of Users')
plt.show()

Here we analyze the catalog coverage of the ALS model:

In [ ]:
# Calculate catalog coverage of ALS model with optimal parameters
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)
model.fit(plays_sparse)
users = list(df.user_id.unique())
catalog = []
for i in range(0,len(users)):
    for x,y in model.recommend(i,plays_sparse.T.tocsr(), N=20, filter_already_liked_items=True):
        if x not in catalog:
            catalog.append(x)
print('Catalog Coverage is', len(catalog)/plays_sparse.shape[1])

## Evaluate and Tune KNN 

Tuning two parameters: K and B. 

For K, we try values [200,400,600,800,1000]

For K1, we stick to default value of 1.2

For B, we try [0,0.5,1]

In [ ]:
# Tune KNN to identify the best parameters
model=implicit.nearest_neighbours.BM25Recommender
max_ndcg_list, heatmap_list = auto_tune_parameter(4,20,model,plays_sparse,[200,400,600,800,1000],1.2,[0,0.5,1])

In [ ]:
# Plot heatmap of parameter tuning results
sns.heatmap(heatmap_list[1], 
            yticklabels=['1000', '800', '600', '400', '200'], 
            xticklabels=[0,0.5,1],
            cbar_kws={'label': 'NDCG Score'})
plt.xlabel('B')
plt.ylabel('Number of Neighbors')
plt.title('KNN Parameter Tuning With NDCG')
plt.show()

In [ ]:
# Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
size = [9000, 20000, 40000, 60000]
ndcg_size_knn = []
recall_size_knn = []
for i in files:
    model = implicit.nearest_neighbours.BM25Recommender(K=400, K1=1.2, B=0)

    #create sparse matrix
    plays_sparse = create_sparse_matrix(i).astype('float')
    print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

    train, test, user_count = split_train_test_per_user(plays_sparse, 4, 20)

    # train model 
    print("Fitting model...")
    model.fit(train, show_progress=True)

    recall, ndcg = evaluate(model, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_size_knn.append(recall)
    ndcg_size_knn.append(ndcg)

Here we analyze the scalability of the KNN model:

In [ ]:
# Plot scalability of KNN model
ndcg_size_knn_df = pd.DataFrame({'N':size,
                       'NDCG': ndcg_size_knn})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_size_knn_df)
plt.title('NDCG by Input Size for KNN')
plt.xlabel('Number Of Users')

Here we analyze the catalog coverage of the KNN model:

In [ ]:
# Calculate catalog coverage of KNN model with optimal parameters
model = implicit.nearest_neighbours.BM25Recommender(K=400, K1=1.2, B=0)
model.fit(plays_sparse)
users = list(df.user_id.unique())
catalog = []
for i in range(0,len(users)):
    for x,y in model.recommend(i,plays_sparse.T.tocsr(), N=20, filter_already_liked_items=True):
        if x not in catalog:
            catalog.append(x)
print('Catalog Coverage is', len(catalog)/plays_sparse.shape[1])